# 보배드림 람다 디테일

In [ ]:
import re
from typing import List, Dict
from datetime import datetime, timezone, timedelta
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
from bs4 import BeautifulSoup

from bobaedream_utils import (
    clean_date_string,
    prepare_for_spark,
    save_html,
)

from common_utils import (
    get_db_connection,
    save_s3_bucket_by_parquet,
    upsert_post_tracking_data,
    get_details_to_parse,
    update_status_banned,
    update_status_failed,
    update_status_changed,
    update_status_unchanged,
    update_changed_stats,
    get_my_ip,
    analyze_post_with_gpt,
    get_proxy_ip,
    return_proxy_ip,
)

# 멀티스레드를 위한 설정
analysis_executor = ThreadPoolExecutor(max_workers=20)

linebreak_ptrn = re.compile(r'(\n){2,}')  # 줄바꿈 문자 매칭

def analyze_post(payload):
    """크롤링된 데이터를 감성 분석 수행"""
    try:
        print(f"🎭 감성 분석 시작: {payload['url']}")
        analyzed_post = analyze_post_with_gpt(payload)
        print(f"✅ 감성 분석 완료: {payload['url']}")
        return analyzed_post
    except Exception as e:
        print(f"❌ 감성 분석 오류: {e}")
        payload['sentiment'] = None
        for comment in payload['comment']:
            comment['sentiment'] = None
        return payload
    
def process_batch(futures: List) -> List[Dict]:
    """배치 단위로 감성 분석 처리"""
    results = []
       
    # 완료된 작업들의 결과를 수집
    for future in as_completed(futures):
        try:
            result = future.result()
            if result:
                results.append(result)
        except Exception as e:
            print(f"Error processing batch item: {e}")
    
    return results


def parse_post_meta(post, post_meta):
    # 포스팅 메타데이터 추가
    title_elem = post_meta.find('dt')
    if title_elem is None:
        print("[ERROR] 제목 파싱 실패.")    
    post['title'] = title_elem['title']    
    count_group = post_meta.find('span', class_='countGroup')
    count_group_em = count_group.find_all('em')
    try:
        view = count_group_em[0].text
        view = int(view.replace(',', ''))  # 쉼표 제거
    except Exception as e:
        print(f"[ERROR] 조회수 파싱 실패: {e}")
        view = -999
    try:
        like = count_group_em[2].text
    except Exception as e:
        print(f"[ERROR] 좋아요 파싱 실패: {e}")
        like = -999
    date_raw = count_group.text
    date_str = date_raw.split('|')[-1].strip()
    try:
        posting_datetime = clean_date_string(date_str)
    except Exception as e:
        print(f"[ERROR] 날짜 파싱 실패: {e}")
        posting_datetime = datetime.strptime('0000-01-01', '%Y-%m-%d')
    post['view'] = view
    post['like'] = like # like 등은 여기서 처리해야 함.        
    post['dislike'] = None
    post['created_at'] = posting_datetime

def parse_detail(total_rows:int = 1):
    # -> Union[List[Dict] | int | None]
    """
    검색 결과의 각 게시물에 대해 상세 정보를 파싱하여 추가합니다.
    
    Args:
        search_results: 검색된 게시물 목록 ([{title, link, ...}, ...])
    
    Returns:
        상세 정보가 추가된 게시물 목록
    """
    lambda_time = time.time()
    headers = {
        # 'User-Agent': UserAgent,
        'Host': 'www.bobaedream.co.kr',
        'Origin': 'https://www.bobaedream.co.kr',
        'Referer': 'https://www.bobaedream.co.kr/search',
        "Accept-Language":"ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Encoding":"gzip, deflate, br, zstd",
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8'
        }
    
    conn = get_db_connection()
    if conn is None:
        print("[ERROR] DB 연결 실패")
        return 500, []
    
    payloads = []
    current_batch = []
    BATCH_SIZE = 50
    PROXY_RETRY_CNT = 5 # 밴 당할 시 프록시로 재접속 시도 횟수
    
    status_code = 200
    table_name = 'probe_bobae'
    while True:
        if time.time() - lambda_time > 810:
            print("[INFO] 람다 함수 시간 초과")
            status_code = 408
            break
        
        # DB에서 상세 정보를 가져올 게시물 목록
        post = get_details_to_parse(conn, table_name, total_rows=total_rows)
        if post is None:
            print("[ERROR] DB 조회 실패")
            status_code = 500
            break
        
        if post == []:
            print("[INFO] 파싱할 게시물이 없습니다.")
            status_code = 204
            break
    
        try:
            post_url = post['url']
            print(f"요청 플랫폼: 보배드림 / '{post_url}' 검색 중...")
            isBanned = False    # 밴 당하면 True
            try:
                response = requests.get(
                        post_url,
                        headers=headers,
                        timeout=10,
                    )
            except Exception as e:
                print(f"[ERROR] 요청 실패: {e}")
                print(f"아마 IP 차단됨! {response.status_code}, 프록시 접근 시도")    
                # 프록시 접근 시도
                isBanned = True
                
            proxy_try_cnt = 0
            while isBanned and proxy_try_cnt > PROXY_RETRY_CNT:
                proxy = get_proxy_ip("bobaedream")
                if proxy is None:
                    # isBanned True
                    break
                try:
                    proxy_try_cnt += 1
                    response = requests.get(
                            post_url,
                            headers=headers,
                            timeout=10,
                            proxies=proxy
                        )
                except Exception as e:
                    print(f"[ERROR] 프록시 요청 실패: {e}")
                else:
                    isBanned = False
                finally:
                    return_proxy_ip(proxy["http"], "bobaedream", isBanned, isTimeout=False)
            
            if isBanned:
                update_status_banned(conn, table_name, post['url'])
                print(f"[ERROR] 페이지 접근 (프록시 사용) 실패")
                continue

            response.encoding = 'utf-8'

            if response.status_code != 200:
                print(f"[ERROR] 확인 필요! status code: {response.status_code}")
                if response.status_code == 403:
                    print(f"IP 차단됨! {response.status_code}")
                    update_status_banned(conn, table_name, post['url'])
                status_code = response.status_code
                break

            # 상세 페이지 요청
            time.sleep(1 + random.random())
                    
            soup = BeautifulSoup(response.text, 'html.parser')
            # 댓글이 없는 경우를 위한 처리
            has_comment = True
            try:
                comment_list = soup.find('div', id='cmt_list').find('ul', class_='basiclist').find_all('li')
                comment_count = len(comment_list)
            except Exception as e:
                print(f"[INFO] 댓글이 없습니다. {e} / post_url: {post_url}")
                has_comment = False
                comment_count = 0
            finally:
                post['comment_count'] = comment_count

            # 본문 내용 파싱
            content_element = soup.find('div', class_='bodyCont')
            try:
                cleaned_content = content_element.text.strip().replace('\xa0', ' ')
                cleaned_content = linebreak_ptrn.sub('\n', cleaned_content)
                post['content'] = cleaned_content
            except Exception as e:
                print(f"[ERROR] 본문 내용 파싱 실패: {e}")
                continue
            try:
                post_meta = soup.find('div', class_='writerProfile').find('dl')
            except Exception as e:
                print(f"[ERROR] 포스팅 메타데이터 파싱 실패: {e}")
                post_meta = None
                continue
            if post_meta is None:
                print("[ERROR] 포스팅 메타데이터 파싱 실패.")
            else:
                # 포스팅 메타데이터 파싱
                parse_post_meta(post, post_meta)
            
            # 댓글 처리
            comment_data = []
            if has_comment:
                for comment in comment_list: # 
                    if "삭제된 댓글입니다" in comment.text:
                        comment_data.append({
                            'created_at': None,
                            'content': None,
                            'like': None,
                            'dislike': None
                        })
                        continue
                    try:
                        comment_meta = comment.find('dl').find('dt').find_all('span')
                    except Exception as e:
                        print(f"[ERROR] 댓글 메타데이터 파싱 실패: {e}")
                        continue
                    
                    if comment_meta is None:
                        print("[ERROR] 댓글 메타데이터 파싱 실패.")
                        continue
                    try:
                        # comment_name = comment_meta[1].text
                        comment_date = datetime.strptime(comment_meta[3].text, '%y.%m.%d %H:%M')
                        comment_content = comment.find('dd').text.strip()
                        comment_like_dislike = comment.find('div', class_='updownbox').find_all('dd')
                        comment_like = comment_like_dislike[0].text.replace('추천 ', '') 
                        comment_dislike = comment_like_dislike[1].text.replace('반대 ', '')
                        comment_data.append({
                            'created_at': comment_date,
                            'content': comment_content,
                            'like': comment_like,
                            'dislike': comment_dislike
                        })
                    except Exception as e:
                        print(f"[ERROR] 댓글 파싱 실패: {e}")
                        continue
            payload = {
                'checked_at': post['checked_at'],
                'platform': 'bobaedream',
                'title': post['title'],
                'post_id': post['post_id'],
                'url': post['url'],
                'content': post['content'],
                'view': post['view'],
                'created_at': post['created_at'],
                'like': post['like'],
                'dislike': post['dislike'],
                'comment_count': post['comment_count'],
                'keywords': post['keywords'],
                'comment': comment_data,
                'status': 'UNCHANGED',
            }       
            
            post['status'] = 'UNCHANGED'
            
            # 모든 포스트에 대해 분석 작업 제출
            future = analysis_executor.submit(analyze_post, payload)
            current_batch.append(future)
            # 완료된 작업들의 결과를 수집
            if len(current_batch) >= BATCH_SIZE:
                print(f"배치 처리 시작 (크기: {len(current_batch)})")
                batch_results = process_batch(current_batch)
                payloads.extend(batch_results)
                current_batch = []

            is_success = update_changed_stats(conn, table_name, post['url'], post['comment_count'], post['view'], post['created_at'])            
            if is_success:
                print(f"[INFO] {post['url']} 업데이트 성공")
            else:
                print(f"[INFO] {post['url']} 업데이트 실패")
            
        except Exception as e:
            post['status'] = 'FAILED'
            payloads.append({
                'status': 'FAILED',
            })
            update_status_failed(conn, table_name, post['url'])
            print(f"[ERROR] {post['url']} 업데이트 실패 / 이유: {e}")    

    # 마지막 배치 처리
    if current_batch:
        print(f"마지막 배치 처리 (크기: {len(current_batch)})")
        batch_results = process_batch(current_batch)
        payloads.extend(batch_results)
    payloads = [payload for payload in payloads if payload['status'] == 'UNCHANGED']
    if payloads:
        print(f"[INFO] 변경된 데이터가 {len(payloads)} 건 있습니다.")
    else:
        status_code = 201
    return status_code, payloads
        

def lambda_handler(event, context):
    # python -m bobaedream.bobaedream_exec 로 실행
    total_rows = event.get("total_rows", 1)
    id = event.get("id")

    get_my_ip()
    table_name = 'probe_bobae'
    status_code, details_data = parse_detail(total_rows)
    if details_data == 500:
        return {
            "status_code": 500,
            "body": "[ERROR] DETAIL / DB 연결 실패"
        }
    elif details_data == []:
        return {
            "status_code": 201,
            "body": "[INFO] DETAIL / S3에 업데이트할 데이터가 없습니다."
        }
    
    try:
        checked_at_dt = details_data[0]['checked_at']
        res = save_s3_bucket_by_parquet(
            checked_at_dt=checked_at_dt,
            platform="bobaedream", 
            data=details_data,
            id=id
        )
        if res == True:
            return {
                "status_code": 200,
                "body": f"[INFO] S3 저장 완료: {len(details_data)} 건"
            }
        else:
            raise
    except Exception as e:
        print(f"[ERROR] S3 저장 실패: {e}")
        conn = get_db_connection()
        if details_data:
            for detail in details_data:
                update_status_changed(conn, table_name, detail["url"])
        return {
            "status_code": 500,
            "body": "[ERROR] S3 저장 실패"
        }
    finally:
        if status_code == 403:
            return {
                "status_code": 403,
                "body": f"[WARNING] DETAIL / IP 차단됨 / 크롤링 데이터: {len(details_data)} 건"
            }
        elif status_code == 408:
            return {
                "status_code": 408,
                "body": f"[WARNING] DETAIL / 람다 함수 시간 초과 / 크롤링 데이터: {len(details_data)} 건"
            }
        

# 보배드림 람다 서치

In [ ]:
import re
from typing import List, Dict, Optional
from datetime import datetime, timezone, timedelta
import time
from urllib.parse import urlencode

import requests
from bs4 import BeautifulSoup

from bobaedream_utils import (
    post_id_salt
)

from common_utils import (
    get_db_connection,
    upsert_post_tracking_data,
    get_my_ip,
    get_proxy_ip,
    return_proxy_ip,
)
linebreak_ptrn = re.compile(r'(\n){2,}')  # 줄바꿈 문자 매칭

def extract_bobaedream(start_date, page_num, keyword) -> Optional[str]:
    print(f"시작 날짜 및 시간: {start_date.strftime('%y.%m.%d')}")
    form_data = {
        "keyword": keyword,
        "colle": "community",
        "searchField": "ALL",
        "page": page_num,
        "sort": "DATE",
        'startDate': start_date.strftime('%y.%m.%d'),
    }
    # data = urlencode(form_data)
    data = form_data
    # 이후 막히면 수정 필요할 수도.
    # UA, cookies, proxies, headers 등을 추가해야 할 수도 있음.
    # 쿠키는  
    # 쿠키는 한 번 selenium으로 로그인해서 받아오면 그걸로 쓰면 됨.
    # 막히면 모바일로도 고려 (touch 등 js 코드에 없어서 모바일로 하면 무조건 가능할 듯).
    
    # UserAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    headers = {
        'Host': 'www.bobaedream.co.kr',
        'Origin': 'https://www.bobaedream.co.kr',
        'Referer': 'https://www.bobaedream.co.kr/search',
        "Accept-Language":"ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Encoding":"gzip, deflate, br, zstd",
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8'
        }
    
    print(f"요청 플랫폼: 보배드림 / 페이지 {page_num}에서 '{keyword}' 검색 중...")
    time.sleep(1)
    response = requests.post(
            'https://www.bobaedream.co.kr/search', 
            data=data,
            headers=headers
        )
    # cookies = response.cookies
    # response_headers = response.headers
    if response.status_code == 200:
        return response.text
    
    print(f"확인 필요! status code: {response.status_code}")
    while response.status_code == 403:
        print(f"[INFO] SEARCH / 보배드림 IP 차단됨! {response.status_code} - 프록시 시도")
        proxy = get_proxy_ip("bobaedream")
        try:
            response = requests.post(
                'https://www.bobaedream.co.kr/search', 
                data=data,
                headers=headers,
                proxies=proxy
            )
        except Exception as e:
            print(f"[INFO] 프록시 오류 발생: {e}")
            return_proxy_ip(proxy["http"], "bobaedream", isBanned=True, isTimeout=True)
        else:
            return_proxy_ip(proxy["http"], "bobaedream", isBanned=response.status_code == 403, isTimeout=False)
    
    if response.status_code == 200:
        return response.text
    else:
        print(f"[WARNING] 기타 오류 / {response.status_code}")
        return None
    
def parse_search(
        html, 
        start_dt: datetime, 
        end_dt: datetime, 
        checked_at: datetime,
        keyword: str
    ) -> Optional[bool]:
    soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')
    
    community_results = soup.find_all('div', class_='search_Community')
    # search_data = []
    if not community_results:
        print('[ERROR] 검색 결과가 없거나, 에러가 발생했습니다.')
        return 404
    
    conn = get_db_connection()
    if conn is None:
        print("[ERROR] DB 연결 실패")
        return 500
    
    table_name = 'probe_bobae'
    for community_result in community_results:
        # ul 태그들 찾기
        lis = community_result.find_all('li')
        if not lis:
            print('[INFO] 검색 결과가 더이상 없습니다.')
            # save_html('htmls/search', html)
            return True

        for li in lis:
            payload = {
                'platform': 'bobaedream',
                'checked_at': checked_at,
            }
            # 각 li 안에서 dt > a 찾기
            try:
                a_tag = li.find('dt').find('a')
                # 타이틀과 url 파싱
                title = a_tag.text.strip()
                url = a_tag['href']
            except Exception as e:
                print(f"dt 혹은 a 태그가 없습니다. : {e}")
                continue
            else:
                payload['title'] = title
                payload['url'] = f"https://www.bobaedream.co.kr{url}"                

            # 각 li 안에서 dd > span 찾기
            try:    
                spans = li.find('dd', class_='path').find_all('span')
                # span 태그 파싱
                if spans[0].text == 'news':
                    print(f"뉴스: {title}, url: {payload['url']}")
                    # continue
                payload['category'] = spans[0].text
                payload['writer'] = spans[1].text
                payload['post_id'] = url.split('No=')[1]
                payload['post_id'] = post_id_salt(payload['post_id'], payload['category'])
                created_at = spans[2].text
            except Exception as e:
                print(f"span 태그가 없습니다. : {e}")
                continue
            else:
                created_at_dt = datetime.strptime(created_at, '%y. %m. %d')
                payload['created_at'] = created_at_dt
                    # 2000년대로 가정
                if created_at_dt.year < 100:
                    created_at_dt = created_at_dt.replace(year=created_at_dt.year + 2000)

                if created_at_dt > end_dt:
                    print(f'[INFO] 기간이 더 뒤이기에 넘어갑니다. {end_dt} / 게시글 날짜: {created_at_dt}')
                    continue
                
                if created_at_dt < start_dt:
                    print(f'[INFO] 기간이 더 앞이기에 종료합니다. {start_dt} / 게시글 날짜: {created_at_dt}')
                    return True

                if payload['category'] == '내차사진':
                    print(f'[WARNING] 내차사진이어서 스킵합니다. {title}')
                    continue
            
            payload['view'] = -999 # 반드시 업데이트되게끔 설정함. # 보배드림 특이 케이스임. (int여야 함!)
            payload['comment_count'] = -999 # 반드시 업데이트되게끔 설정함. # 보배드림 특이 케이스임. (int여야 함!)
            payload['status'] = 'CHANGED'  # 반드시 업데이트되게끔 설정함.
            payload['keyword'] = keyword
            # DB에 변경사항 저장
            # comment_count, view를 확인할 수 있으면 바로 업데이트.
            upsert_post_tracking_data(
                    conn=conn,
                    table_name=table_name,
                    payload=payload
                )

def lambda_handler(event, context):
    """
    keyword, checked_at 필수.
    start_date, 
    end_date는 선택.
    """
    #TODO: 무조건 바꿔야 함! (end_date 이상 넘어가면 안되게끔.)
    # 이게 크롤링한 시간.
    checked_at_str = event.get('checked_at')
    # ISO 8601 형식 → UTC 기준
    if checked_at_str:
        event_time = datetime.fromisoformat(checked_at_str)
    else:
        event_time = datetime.now(timezone.utc).replace(tzinfo=None)  # 명시적으로 UTC 시간을 가져온 후 timezone 정보 제거
    # UTC 시간에 9시간을 더해 KST 시간으로 변환
    checked_at = event_time + timedelta(hours=9)  # UTC+9 (KST)
    # KST 시간 출력 (형식: ‘YYYY-MM-DD HH:MM:SS’)
    print("한국 시간:", checked_at.strftime('%Y-%m-%d %H:%M:%S'))

    # 게시글 시작 날짜
    start_date = event.get('start_date')
    if start_date is None:
        start_dt = checked_at - timedelta(days=14)
    else:
        start_dt = datetime.strptime(start_date, '%Y-%m-%d')
        # start_dt = start_dt.replace(tzinfo=timezone.utc)  # UTC로 변환
    
    # 게시글 종료 날짜
    end_date = event.get('end_date')
    if end_date is None:
        end_dt = checked_at + timedelta(days=0)
    else:
        end_dt = datetime.strptime(end_date, '%Y-%m-%d')
        # end_dt = end_dt.replace(tzinfo=timezone.utc)
    
    get_my_ip()
    
    # 검색할 키워드
    keyword = event.get('keyword')
    for i in range(1, 1000):
        html = extract_bobaedream(start_date, page_num=i, keyword=keyword)
        # save_html('htmls/search', html)
        if html is None:
            return {
                "status_code": 403,
                "body": "[WARNING] SEARCH / 보배드림(platform으로 대체) IP 차단됨!"
            }
        result = parse_search(html, start_dt=start_dt, end_dt=end_dt, checked_at=checked_at, keyword=keyword)   
        if result == 404:
            print(f"[INFO] 검색 결과가 없습니다. {keyword}")
        
        if result == 500:
            print(f"[ERROR] DB 연결 실패: {keyword}")
            return {
                "status_code": 500,
                "body": "[ERROR] SEARCH / DB 연결 실패"
            }
            
        if result == True:
            print(f"[INFO] 보배드림 검색 종료: {keyword}")
            return {
                "status_code": 200,
                "body":  "[INFO] SEARCH / DB 업데이트 성공"
            }

In [3]:
a = {"this": "test", "aaa": "bbb"}
print (a)
print (len (a))

{'this': 'test', 'aaa': 'bbb'}
2
